<h1> Data Spaces' Tesina <h1>

<p>Imports section<p>

In [1]:
import numpy as np
from numpy import genfromtxt
from sklearn import neighbors, model_selection, metrics, preprocessing

<p> Read .csv file <p>

In [2]:
filename = "dataR2.csv"
file = open(filename, "r")
# data = np.genfromtxt(file, delimiter=",", dtype=None, encoding=None, names=True, usecols=(0,1,2,3,4,5,6,7,8,9))
# method above did not work properly
data = np.loadtxt(file, delimiter=",", dtype=None, encoding=None, usecols=(0,1,2,3,4,5,6,7,8,9), skiprows=1)

# records with the classification as "1" are Healthy Controls, "2" means Patients.

<p>Preprocessing</p>

In [3]:
# Let's divide data into source and target (respectively X and Y)
X = data[:, :-1]
Y = data[:, len(data[0])-1]

X_train, X_test, Y_train, Y_test = model_selection.train_test_split(X, Y, random_state=np.random.randint(0,100), test_size=0.3)

# array of possible K to apply KNN neighbors
ks = [3,5,7,9]


# normalize data because algorithms work better with normalized data
scaler = preprocessing.StandardScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

<p>Apply K-NN</p>

In [4]:
for k in ks:
    n_neighbors = k
    
    # Create an instance of neighbors classifier (clf) and fit the data
    clf = neighbors.KNeighborsClassifier(n_neighbors)
    
    # train the classifier on the training set
    clf.fit(X_train, Y_train)
    
    print("Accuracy score on the test set with K =",n_neighbors,"is %.3f" %(clf.score(X_test, Y_test)))


    

Accuracy score on the test set with K = 3 is 0.657
Accuracy score on the test set with K = 5 is 0.600
Accuracy score on the test set with K = 7 is 0.600
Accuracy score on the test set with K = 9 is 0.629


<p>Apply KNN this time with a validation set </p>

In [5]:
# Let's create the validation set
X_train_t, X_valid, Y_train_t, Y_valid = model_selection.train_test_split(X_train, Y_train, random_state=np.random.randint(0,100), test_size=0.15)

# array of possible K to apply KNN neighbors
ks = [3,5,7,9]

# normalize data because algorithms work better with normalized data
scaler = preprocessing.StandardScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)
X_valid = scaler.transform(X_valid)

# array to write accuracy values for each K 
acc_arr = []

for k in ks:
    n_neighbors = k
    
    # Create an instance of neighbors classifier (clf) and fit the data
    clf = neighbors.KNeighborsClassifier(n_neighbors)
    
    # train the classifier on the training set
    clf.fit(X_train_t, Y_train_t)
    
    acc_arr.append(clf.score(X_valid, Y_valid))
    
# I choose the best K based on the results on the validation set and apply KNN with that
k_best_index = acc_arr.index(max(acc_arr))
k_best = ks[k_best_index]

clf2 = neighbors.KNeighborsClassifier(k_best)
clf2.fit(X_train_t, Y_train_t)

print("The best K, based on the validation set results, is",k_best,"and the accuracy on the test set is %.3f"%(clf2.score(X_test, Y_test)))

The best K, based on the validation set results, is 3 and the accuracy on the test set is 0.686
